In [1]:
import os
import xmltodict
import pandas as pd
import numpy as np
from html.parser import HTMLParser  # Python 3
dirname = os.path.join('data', 'Bayzick')

messages = pd.DataFrame()
consensus = pd.DataFrame()
print("Running....")
num_files = len(os.listdir(dirname))
num_errors = 0
for filename in os.listdir(dirname):
    if filename in [os.listdir(dirname)[7], os.listdir(dirname)[8], os.listdir(dirname)[10], os.listdir(dirname)[15]]:
        continue
    if '.xml' in filename:
        try:
            file = dirname + '/' + filename
            messages_df = pd.DataFrame(pd.DataFrame(xmltodict.parse(open(file, "rb")))['posts'][0])
            messages_df['origin'] = filename.strip('.xml')
            messages = pd.concat([messages, messages_df])
        except:
            num_errors += 1
            print("error in {}".format(file))
    elif '.xlsx' in filename:
        file = dirname + '/' + filename
        # We need to ignore the first 2 lines of these .xlsx files
        consensus = pd.concat([consensus, pd.DataFrame(pd.read_excel(open(file, 'rb'),converters={'File Name':str}, skiprows=[0,1]))])

print(messages[['body', 'origin']])
print(num_errors, num_files)

Running....
error in data/Bayzick/3991200.0005.xml
error in data/Bayzick/44762.0002.xml
error in data/Bayzick/3952761.0000.xml
error in data/Bayzick/3889364.0000.xml
error in data/Bayzick/44762.0016.xml
error in data/Bayzick/4013964.0000.xml
error in data/Bayzick/4070823.0011.xml
error in data/Bayzick/1734409.0000.xml
error in data/Bayzick/1734409.0001.xml
error in data/Bayzick/4257939.0000.xml
error in data/Bayzick/4070823.0010.xml
error in data/Bayzick/44762.0017.xml
error in data/Bayzick/44762.0003.xml
error in data/Bayzick/3991200.0004.xml
error in data/Bayzick/4101671.0009.xml
error in data/Bayzick/4101671.0035.xml
error in data/Bayzick/4104112.0017.xml
error in data/Bayzick/4255475.0014.xml
error in data/Bayzick/4255475.0016.xml
error in data/Bayzick/4104112.0015.xml
error in data/Bayzick/4071277.0009.xml
error in data/Bayzick/3991200.0006.xml
error in data/Bayzick/44762.0015.xml
error in data/Bayzick/44762.0001.xml
error in data/Bayzick/4070823.0012.xml
error in data/Bayzick/173

error in data/Bayzick/4176028.0016.xml
error in data/Bayzick/6880.0011.xml
error in data/Bayzick/4114208.0011.xml
error in data/Bayzick/4114208.0005.xml
error in data/Bayzick/1789562.0026.xml
error in data/Bayzick/1789562.0032.xml
error in data/Bayzick/1689916.0029.xml
error in data/Bayzick/4255475.0020.xml
error in data/Bayzick/4101671.0029.xml
error in data/Bayzick/3834378.0004.xml
error in data/Bayzick/3933486.0000.xml
error in data/Bayzick/4157225.0000.xml
error in data/Bayzick/4070823.0018.xml
error in data/Bayzick/4070823.0024.xml
error in data/Bayzick/1921204.0000.xml
error in data/Bayzick/4092918.0019.xml
error in data/Bayzick/4092918.0018.xml
error in data/Bayzick/4092918.0024.xml
error in data/Bayzick/1921204.0001.xml
error in data/Bayzick/4070823.0019.xml
error in data/Bayzick/1729363.0038.xml
error in data/Bayzick/3834378.0005.xml
error in data/Bayzick/4101671.0028.xml
error in data/Bayzick/4255475.0021.xml
error in data/Bayzick/4255475.0009.xml
error in data/Bayzick/425547

In [3]:
import pickle
pickle.dump(messages, open("out.pkl", 'wb'))
#messages.groupby(["origin"])["body"].transform(lambda x: " ".join(str(x))).drop_duplicates()

In [4]:
consensus = consensus.rename(columns={"File Name": "origin"})
print(consensus.columns)

Index(['origin', 'Is Cyberbullying Present?', 'Unnamed: 2', 'Unnamed: 3'], dtype='object')


In [5]:
messages = messages.merge(consensus)

In [12]:
df_text_label = messages[["body","Is Cyberbullying Present?"]]
label2num = lambda x : 1 if x == "Y" else 0
lowercase = lambda x : str(x).lower()

# TODO Figure out a more elegant way to do this... (without warning)
df_text_label["Is Cyberbullying Present?"] = df_text_label["Is Cyberbullying Present?"].apply(label2num)
df_text_label["body"] = df_text_label["body"].apply(lowercase)

df_text_label.head()

/Users/jasrajjohl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jasrajjohl/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,body,Is Cyberbullying Present?
0,lol..typical paranoid obamaniacs.,0
1,lol as a black man in america i embrace my par...,0
2,a good question to obama supporters and especi...,0
3,"i would vote for any democrat, i dont know any...",0
4,it's pretty obvious they are going to try and ...,0


In [11]:
OUT_FILENAME = "final_data/bayzick_clean.csv"
df_text_label.to_csv(OUT_FILENAME, index=False)

In [8]:
len(df_text_label)

15325

In [9]:
OUT_UNLABEDED = "final_data/bayzick_unlabeled.csv"
df_text_label["body"].to_csv(OUT_UNLABEDED, index=False, header=False)

In [10]:
df_text_label["body"][2]

'a good question to obama supporters and especially the black voters. would you support hillary clinton if she was chosen to be the nominee by the superdelegates?'